# Phase 1 Ablation Study: Pure PPO vs GNS vs PhysRobot-SV

**Self-contained** Colab notebook for the PhysRobot paper ablation.

- 3 methods: Pure PPO (~10K), GNS (~5K), PhysRobot-SV (~6K physics + policy)
- 500K timesteps x 5 seeds per method
- OOD mass sweep: [0.5, 0.75, 1.0, 1.5, 2.0]
- All parameters from SPEC.md (single source of truth)


In [ ]:
# Cell 1: Install dependencies
%%time
!pip install mujoco gymnasium stable-baselines3[extra] torch torch-geometric matplotlib pandas -q
import torch
print(f'PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')


In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/physrobot_phase1_ablation'
for sub in ['', 'models', 'results', 'logs', 'figures']:
    os.makedirs(f'{SAVE_DIR}/{sub}' if sub else SAVE_DIR, exist_ok=True)
print(f'Save dir: {SAVE_DIR}')


In [ ]:
# Cell 3: PushBoxEnv (16-dim obs, 2-dim action, per SPEC.md)
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

XML = '''<mujoco model="push_box">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option timestep="0.002" integrator="Euler" gravity="0 0 -9.81">
    <flag warmstart="enable"/>
  </option>
  <visual>
    <global offwidth="1280" offheight="720"/>
    <quality shadowsize="4096"/>
    <map force="0.1" zfar="30"/>
  </visual>
  <asset>
    <texture builtin="gradient" height="100" rgb1="0.3 0.5 0.7" rgb2="0.1 0.2 0.3" type="skybox" width="100"/>
    <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
    <texture builtin="checker" height="100" name="texplane" rgb1="0.2 0.2 0.2" rgb2="0.3 0.3 0.3" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.3" shininess="0.5" specular="0.5" texrepeat="3 3" texture="texplane"/>
    <material name="geom" texture="texgeom" texuniform="true"/>
  </asset>
  <default>
    <joint armature="0.01" damping="0.1" limited="true"/>
    <geom conaffinity="1" condim="3" contype="1" friction="0.5 0.005 0.0001" margin="0.001" material="geom" rgba="0.8 0.6 0.4 1"/>
  </default>
  <worldbody>
    <light directional="true" diffuse="0.8 0.8 0.8" pos="0 0 3" dir="0 0 -1"/>
    <light directional="true" diffuse="0.4 0.4 0.4" pos="0 0 3" dir="1 1 -1"/>
    <geom name="floor" type="plane" size="3 3 0.1" rgba="0.8 0.8 0.8 1" material="MatPlane"/>
    <body name="arm_base" pos="0 0 0.02">
      <geom name="base_geom" type="cylinder" size="0.05 0.02" rgba="0.3 0.3 0.3 1"/>
      <body name="upper_arm" pos="0 0 0.02">
        <joint name="shoulder" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
        <geom name="upper_arm_geom" type="capsule" fromto="0 0 0 0.4 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
        <body name="forearm" pos="0.4 0 0">
          <joint name="elbow" type="hinge" axis="0 0 1" range="-180 180" damping="0.5"/>
          <geom name="forearm_geom" type="capsule" fromto="0 0 0 0.3 0 0" size="0.025" rgba="0.5 0.5 0.8 1"/>
          <site name="endeffector" pos="0.3 0 0" size="0.03" rgba="1 0.5 0 0.8"/>
        </body>
      </body>
    </body>
    <body name="box" pos="0.35 0 0.05">
      <freejoint name="box_freejoint"/>
      <geom name="box_geom" type="box" size="0.05 0.05 0.05" mass="0.5" rgba="0.2 0.8 0.2 1" friction="0.5 0.005 0.0001"/>
      <site name="box_center" pos="0 0 0" size="0.01" rgba="0 1 0 1"/>
    </body>
    <site name="goal" pos="0.5 0.3 0.02" size="0.06" rgba="1 0 0 0.4" type="sphere"/>
  </worldbody>
  <actuator>
    <motor name="shoulder_motor" joint="shoulder" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
    <motor name="elbow_motor" joint="elbow" gear="1.0" ctrllimited="true" ctrlrange="-10 10"/>
  </actuator>
</mujoco>
'''


class PushBoxEnv(gym.Env):
    """
    PushBox environment per SPEC.md:
      - 16-dim obs: [joint_pos(2), joint_vel(2), ee_pos(3), box_pos(3), box_vel(3), goal_pos(3)]
      - 2-dim action: [shoulder_torque, elbow_torque] in [-10, 10] Nm
      - Success threshold: 0.15 m
      - Episode length: 500 steps
      - 5 MuJoCo substeps per control step (dt_ctrl = 0.01 s)
    """
    def __init__(self, render_mode=None, box_mass=0.5):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_string(XML)
        self.data = mujoco.MjData(self.model)
        self.box_mass = box_mass
        self._set_box_mass(box_mass)
        self._ee_site_id = mujoco.mj_name2id(
            self.model, mujoco.mjtObj.mjOBJ_SITE, 'endeffector'
        )
        self.action_space = spaces.Box(
            low=-10.0, high=10.0, shape=(2,), dtype=np.float32
        )
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(16,), dtype=np.float32
        )
        self.goal_pos = np.array([0.5, 0.3, 0.02])
        self.max_episode_steps = 500
        self.current_step = 0
        self.success_threshold = 0.15
        self._prev_dist_box_goal = None
        self._prev_dist_ee_box = None
        self.render_mode = render_mode

    def _set_box_mass(self, mass):
        box_body_id = mujoco.mj_name2id(
            self.model, mujoco.mjtObj.mjOBJ_BODY, 'box'
        )
        self.model.body_mass[box_body_id] = mass

    def set_box_mass(self, mass):
        self.box_mass = mass
        self._set_box_mass(mass)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        if seed is not None:
            np.random.seed(seed)
        self.data.qpos[0] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[1] = np.random.uniform(-0.5, 0.5)
        self.data.qpos[2] = np.random.uniform(0.25, 0.45)
        self.data.qpos[3] = np.random.uniform(-0.15, 0.15)
        self.data.qpos[4] = 0.05
        self.data.qpos[5:9] = [1, 0, 0, 0]
        self.data.qvel[:] = 0.0
        mujoco.mj_forward(self.model, self.data)
        self.current_step = 0
        ee_pos = self.data.site_xpos[self._ee_site_id]
        box_pos = self.data.qpos[2:5]
        self._prev_dist_box_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        self._prev_dist_ee_box = np.linalg.norm(ee_pos[:2] - box_pos[:2])
        return self._get_obs(), self._get_info()

    def _get_obs(self):
        joint_pos = self.data.qpos[:2].copy()
        joint_vel = self.data.qvel[:2].copy()
        ee_pos = self.data.site_xpos[self._ee_site_id].copy()
        box_pos = self.data.qpos[2:5].copy()
        box_vel = self.data.qvel[2:5].copy()
        goal_pos = self.goal_pos.copy()
        obs = np.concatenate([joint_pos, joint_vel, ee_pos, box_pos, box_vel, goal_pos])
        return obs.astype(np.float32)

    def _get_info(self):
        box_pos = self.data.qpos[2:5]
        dist = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])
        return {
            'distance_to_goal': dist,
            'success': dist < self.success_threshold,
            'box_mass': self.box_mass,
            'timestep': self.current_step,
        }

    def step(self, action):
        self.data.ctrl[:] = action
        for _ in range(5):
            mujoco.mj_step(self.model, self.data)

        ee_pos = self.data.site_xpos[self._ee_site_id].copy()
        box_pos = self.data.qpos[2:5].copy()

        dist_ee_box = np.linalg.norm(ee_pos[:2] - box_pos[:2])
        dist_box_goal = np.linalg.norm(box_pos[:2] - self.goal_pos[:2])

        # Progress-based reward (SPEC.md / week1_v2)
        reach_progress = self._prev_dist_ee_box - dist_ee_box
        self._prev_dist_ee_box = dist_ee_box
        push_progress = self._prev_dist_box_goal - dist_box_goal
        self._prev_dist_box_goal = dist_box_goal

        reward = (
            0.5 * (-dist_ee_box)
            + 1.0 * (-dist_box_goal)
            + 10.0 * reach_progress
            + 20.0 * push_progress
            - 0.01 * np.sum(action ** 2)
        )

        success = dist_box_goal < self.success_threshold
        if success:
            remaining = self.max_episode_steps - self.current_step
            reward += 500.0 + remaining * 1.0

        self.current_step += 1
        terminated = success
        truncated = self.current_step >= self.max_episode_steps
        return self._get_obs(), reward, terminated, truncated, self._get_info()

    def render(self):
        pass

    def close(self):
        pass


def make_push_box_env(box_mass=0.5):
    def _init():
        return PushBoxEnv(box_mass=box_mass)
    return _init


# Quick sanity check
env = PushBoxEnv()
obs, info = env.reset()
assert obs.shape == (16,), f'Expected 16-dim obs, got {obs.shape}'
assert env.action_space.shape == (2,)
print(f'Obs: {obs.shape}, Action: {env.action_space.shape}, Threshold: {env.success_threshold}m')
env.close()
print('[OK] PushBoxEnv ready')


In [ ]:
# Cell 4: Agent definitions (Pure PPO, GNS, PhysRobot-SV)
import torch
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure
import json
import time

try:
    from torch_geometric.nn import MessagePassing
    from torch_geometric.data import Data as PyGData, Batch as PyGBatch
    HAS_PYG = True
except ImportError:
    HAS_PYG = False
    print('[WARN] torch_geometric not found; GNS will use fallback MLP.')

# ---------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------
EPS = 1e-7
DEG_EPS = 1e-4


def _make_mlp(in_dim, hidden_dim, out_dim):
    """2-layer MLP with LayerNorm + ReLU (from sv_message_passing.py)."""
    return nn.Sequential(
        nn.Linear(in_dim, hidden_dim),
        nn.LayerNorm(hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, out_dim),
    )


def count_params(module):
    return sum(p.numel() for p in module.parameters())


# ---------------------------------------------------------------
# Logging callback: records per-episode reward + success
# ---------------------------------------------------------------
class LoggingCallback(BaseCallback):
    """Tracks episode rewards and successes for learning curves."""
    def __init__(self):
        super().__init__()
        self.ep_rewards = []
        self.ep_successes = []
        self.ep_timesteps = []
        self._current_rewards = None

    def _on_training_start(self):
        n_envs = self.training_env.num_envs
        self._current_rewards = [0.0] * n_envs

    def _on_step(self):
        rewards = self.locals.get('rewards', [])
        dones = self.locals.get('dones', [])
        infos = self.locals.get('infos', [{}])
        for i in range(len(dones)):
            if i < len(rewards):
                self._current_rewards[i] += rewards[i]
            if dones[i]:
                self.ep_rewards.append(self._current_rewards[i])
                info = infos[i] if i < len(infos) else {}
                self.ep_successes.append(1 if info.get('success', False) else 0)
                self.ep_timesteps.append(self.num_timesteps)
                self._current_rewards[i] = 0.0
        return True


# ===============================================================
# AGENT 1: Pure PPO  (~10K params, [64,64] MLP pi + vf)
# ===============================================================
# Uses SB3 MlpPolicy with net_arch=dict(pi=[64,64], vf=[64,64])
# No features extractor beyond default flatten.


# ===============================================================
# AGENT 2: GNS  (~5K params, d_h=32, L=1)
# ===============================================================

def obs_to_graph_batch(observations):
    """Convert [B,16] obs -> PyG Batch with 2 nodes (ee, box)."""
    B = observations.shape[0]
    dev = observations.device
    graphs = []
    for i in range(B):
        o = observations[i]
        ee_pos = o[4:7]
        ee_vel = torch.zeros(3, device=dev)
        box_pos = o[7:10]
        box_vel = o[10:13]
        positions = torch.stack([ee_pos, box_pos])
        velocities = torch.stack([ee_vel, box_vel])
        node_feats = torch.cat([positions, velocities], dim=-1)  # [2,6]
        edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long, device=dev).t().contiguous()
        rel01 = box_pos - ee_pos
        rel10 = ee_pos - box_pos
        d01 = torch.norm(rel01).unsqueeze(0)
        d10 = torch.norm(rel10).unsqueeze(0)
        edge_attr = torch.stack([torch.cat([rel01, d01]), torch.cat([rel10, d10])])
        g = PyGData(x=node_feats, pos=positions, edge_index=edge_index, edge_attr=edge_attr)
        graphs.append(g)
    return PyGBatch.from_data_list(graphs)


if HAS_PYG:
    class GNSGraphLayerV2(MessagePassing):
        """Single GN message-passing layer. ~2K params."""
        def __init__(self, node_dim, edge_dim, hidden_dim=32):
            super().__init__(aggr='add')
            self.edge_mlp = nn.Sequential(
                nn.Linear(2 * node_dim + edge_dim, hidden_dim), nn.ReLU(),
                nn.Linear(hidden_dim, edge_dim),
            )
            self.node_mlp = nn.Sequential(
                nn.Linear(node_dim + edge_dim, hidden_dim), nn.ReLU(),
                nn.Linear(hidden_dim, node_dim),
            )

        def forward(self, x, edge_index, edge_attr):
            return self.propagate(edge_index, x=x, edge_attr=edge_attr)

        def message(self, x_i, x_j, edge_attr):
            return self.edge_mlp(torch.cat([x_i, x_j, edge_attr], dim=-1))

        def update(self, aggr_out, x):
            return self.node_mlp(torch.cat([x, aggr_out], dim=-1))

    class GNSFeaturesExtractorV2(BaseFeaturesExtractor):
        """GNS extractor: node_enc -> 1x GNSGraphLayer -> decoder -> proj. ~5K params."""
        def __init__(self, observation_space, features_dim=64):
            super().__init__(observation_space, features_dim)
            hid = 32
            edge_dim = 4
            self.node_encoder = nn.Sequential(nn.Linear(6, hid), nn.ReLU())
            self.edge_encoder = nn.Sequential(nn.Linear(edge_dim, hid), nn.ReLU())
            self.gn_layer = GNSGraphLayerV2(hid, hid, hid)
            self.decoder = nn.Linear(hid, 3)
            self.feature_proj = nn.Sequential(
                nn.Linear(3 + 16, features_dim), nn.ReLU(),
            )

        def forward(self, observations):
            graph = obs_to_graph_batch(observations)
            x = self.node_encoder(graph.x)
            ea = self.edge_encoder(graph.edge_attr)
            x = x + self.gn_layer(x, graph.edge_index, ea)
            acc = self.decoder(x)
            box_acc = acc[1::2]
            combined = torch.cat([box_acc, observations], dim=-1)
            return self.feature_proj(combined)


# ===============================================================
# AGENT 3: PhysRobot-SV  (SV message passing from sv_message_passing.py)
# ===============================================================

def build_edge_frames(pos, vel, src, dst):
    """
    Antisymmetric edge-local ONB {e1, e2, e3}.
    From sv_message_passing.py L57-112.
    """
    r_ij = pos[dst] - pos[src]
    d_ij = torch.norm(r_ij, dim=-1, keepdim=True)
    e1 = r_ij / (d_ij + EPS)

    v_rel = vel[dst] - vel[src]
    v_par = (v_rel * e1).sum(dim=-1, keepdim=True) * e1
    v_perp = v_rel - v_par
    v_perp_norm = torch.norm(v_perp, dim=-1, keepdim=True)

    non_degenerate = (v_perp_norm > DEG_EPS).float()
    e2_vel = v_perp / (v_perp_norm + EPS)

    z_hat = torch.tensor([0.0, 0.0, 1.0], device=pos.device).expand_as(e1)
    e2_fall_raw = torch.cross(e1, z_hat, dim=-1)
    e2_fall_norm = torch.norm(e2_fall_raw, dim=-1, keepdim=True)
    use_y = (e2_fall_norm < DEG_EPS).float()
    y_hat = torch.tensor([0.0, 1.0, 0.0], device=pos.device).expand_as(e1)
    e2_fall_raw2 = torch.cross(e1, y_hat, dim=-1)
    e2_fall_raw = (1 - use_y) * e2_fall_raw + use_y * e2_fall_raw2
    e2_fall_norm = torch.norm(e2_fall_raw, dim=-1, keepdim=True)
    e2_fall = e2_fall_raw / (e2_fall_norm + EPS)

    e2 = non_degenerate * e2_vel + (1 - non_degenerate) * e2_fall
    e3 = torch.cross(e1, e2, dim=-1)

    return e1, e2, e3, r_ij, d_ij


class SVMessagePassing(nn.Module):
    """
    One round of SV message passing (undirected-pair approach).
    Hard-codes Newton's 3rd law: +F to j, -F to i.
    From sv_message_passing.py L110-234.
    """
    def __init__(self, node_dim, hidden_dim=32):
        super().__init__()
        self.node_dim = node_dim
        self.hidden_dim = hidden_dim
        n_scalar = 5  # d, v_r, v_t, v_b, ||v||
        self.force_mlp = _make_mlp(
            in_dim=n_scalar + 2 * node_dim,
            hidden_dim=hidden_dim,
            out_dim=3,
        )
        self.node_update = _make_mlp(
            in_dim=node_dim + 3,
            hidden_dim=hidden_dim,
            out_dim=node_dim,
        )

    def _extract_undirected_pairs(self, edge_index):
        src, dst = edge_index[0], edge_index[1]
        mask = src < dst
        return torch.stack([src[mask], dst[mask]], dim=0)

    def forward_with_forces(self, h, edge_index, pos, vel):
        N = h.size(0)
        pairs = self._extract_undirected_pairs(edge_index)
        pi, pj = pairs[0], pairs[1]

        e1, e2, e3, r_ij, d_ij = build_edge_frames(pos, vel, pi, pj)
        v_rel = vel[pj] - vel[pi]

        v_r = (v_rel * e1).sum(dim=-1, keepdim=True)
        v_t = (v_rel * e2).sum(dim=-1, keepdim=True)
        v_b = (v_rel * e3).sum(dim=-1, keepdim=True)
        v_norm = torch.norm(v_rel, dim=-1, keepdim=True)

        scalars_geom = torch.cat([d_ij, v_r, v_t, v_b, v_norm], dim=-1)
        h_sum = h[pi] + h[pj]
        h_diff_abs = (h[pi] - h[pj]).abs()
        scalars = torch.cat([scalars_geom, h_sum, h_diff_abs], dim=-1)

        alphas = self.force_mlp(scalars)
        alpha1, alpha2, alpha3 = alphas[:, 0:1], alphas[:, 1:2], alphas[:, 2:3]
        force_ij = alpha1 * e1 + alpha2 * e2 + alpha3 * e3

        F_agg = torch.zeros(N, 3, device=h.device, dtype=h.dtype)
        F_agg.scatter_add_(0, pj.unsqueeze(-1).expand_as(force_ij), force_ij)
        F_agg.scatter_add_(0, pi.unsqueeze(-1).expand_as(force_ij), -force_ij)

        h_input = torch.cat([h, F_agg], dim=-1)
        h_new = h + self.node_update(h_input)
        return h_new, F_agg


class SVPhysicsCore(nn.Module):
    """
    Physics stream: encoder -> L x SVMessagePassing -> forces.
    PushBox config: node_input=6, d_h=32, L=1. ~6K params.
    From sv_message_passing.py L242-314.
    """
    def __init__(self, node_input_dim=6, hidden_dim=32, n_layers=1):
        super().__init__()
        self.encoder = _make_mlp(node_input_dim, hidden_dim, hidden_dim)
        self.sv_layers = nn.ModuleList([
            SVMessagePassing(node_dim=hidden_dim, hidden_dim=hidden_dim)
            for _ in range(n_layers)
        ])

    def forward(self, positions, velocities, edge_index):
        node_features = torch.cat([positions, velocities], dim=-1)
        h = self.encoder(node_features)
        forces = None
        for layer in self.sv_layers:
            h, F_agg = layer.forward_with_forces(h, edge_index, positions, velocities)
            forces = F_agg
        return forces


class PhysRobotSVExtractor(BaseFeaturesExtractor):
    """
    Dual-stream extractor with real SV message passing.
    Policy stream: MLP 16->64->64 (ReLU)
    Physics stream: SVPhysicsCore (d_h=32, L=1)
    Fusion: Linear(64+3, 64) + ReLU, with stop-gradient on physics.
    From sv_message_passing.py L340-421.
    """
    def __init__(self, observation_space, features_dim=64):
        super().__init__(observation_space, features_dim)
        self.physics_core = SVPhysicsCore(
            node_input_dim=6, hidden_dim=32, n_layers=1,
        )
        self.policy_stream = nn.Sequential(
            nn.Linear(16, 64), nn.ReLU(),
            nn.Linear(64, features_dim), nn.ReLU(),
        )
        self.fusion = nn.Sequential(
            nn.Linear(features_dim + 3, features_dim), nn.ReLU(),
        )
        self._edge_index_2 = torch.tensor(
            [[0, 1], [1, 0]], dtype=torch.long
        ).t()

    def _obs_to_graph(self, obs):
        ee_pos = obs[:, 4:7]
        box_pos = obs[:, 7:10]
        box_vel = obs[:, 10:13]
        ee_vel = torch.zeros_like(ee_pos)
        return ee_pos, ee_vel, box_pos, box_vel

    def forward(self, observations):
        B = observations.shape[0]
        device = observations.device
        z_policy = self.policy_stream(observations)
        ee_pos, ee_vel, box_pos, box_vel = self._obs_to_graph(observations)
        edge_index = self._edge_index_2.to(device)
        box_acc_list = []
        for i in range(B):
            pos_i = torch.stack([ee_pos[i], box_pos[i]], dim=0)
            vel_i = torch.stack([ee_vel[i], box_vel[i]], dim=0)
            acc_i = self.physics_core(pos_i, vel_i, edge_index)
            box_acc_list.append(acc_i[1])  # box node
        z_physics = torch.stack(box_acc_list, dim=0)
        z_physics_sg = z_physics.detach()  # stop-gradient
        combined = torch.cat([z_policy, z_physics_sg], dim=-1)
        return self.fusion(combined)


# ---------------------------------------------------------------
# Param count verification
# ---------------------------------------------------------------
_tmp_env = DummyVecEnv([make_push_box_env()])

# Pure PPO
_ppo = PPO('MlpPolicy', _tmp_env, policy_kwargs=dict(
    net_arch=dict(pi=[64, 64], vf=[64, 64])), verbose=0)
print(f'Pure PPO policy params: {count_params(_ppo.policy):>8,}')

# GNS
if HAS_PYG:
    _gns = PPO('MlpPolicy', _tmp_env, policy_kwargs=dict(
        features_extractor_class=GNSFeaturesExtractorV2,
        features_extractor_kwargs=dict(features_dim=64),
        net_arch=dict(pi=[64, 64], vf=[64, 64])), verbose=0)
    print(f'GNS policy params:      {count_params(_gns.policy):>8,}')
    del _gns

# PhysRobot-SV
_sv = PPO('MlpPolicy', _tmp_env, policy_kwargs=dict(
    features_extractor_class=PhysRobotSVExtractor,
    features_extractor_kwargs=dict(features_dim=64),
    net_arch=dict(pi=[64, 64], vf=[64, 64])), verbose=0)
print(f'PhysRobot-SV params:    {count_params(_sv.policy):>8,}')

# Physics core alone
_phys = SVPhysicsCore(node_input_dim=6, hidden_dim=32, n_layers=1)
print(f'  (physics core only:   {count_params(_phys):>8,})')

del _ppo, _sv, _phys
_tmp_env.close()
print('[OK] All 3 agents defined')


In [ ]:
# Cell 5: Training configuration (SPEC.md)
METHODS = ['Pure_PPO', 'GNS', 'PhysRobot_SV']
SEEDS = [42, 123, 256, 789, 1024]
TOTAL_TIMESTEPS = 500_000
N_ENVS = 4
BOX_MASS_TRAIN = 0.5  # default training mass
EVAL_EPISODES = 100
EVAL_FREQ = 50_000  # evaluate every 50K steps
OOD_MASSES = [0.5, 0.75, 1.0, 1.5, 2.0]

# PPO hyperparams (SPEC.md Section 3)
PPO_KWARGS = dict(
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=0,
)

print('=== Phase 1 Ablation Config ===')
print(f'Methods:    {METHODS}')
print(f'Seeds:      {SEEDS}')
print(f'Timesteps:  {TOTAL_TIMESTEPS:,}')
print(f'Envs:       {N_ENVS}')
print(f'OOD masses: {OOD_MASSES}')
est_min = len(METHODS) * len(SEEDS) * TOTAL_TIMESTEPS / 200_000 * 5
print(f'Est. time:  ~{est_min:.0f} min on T4')


In [ ]:
# Cell 6: Training loop (3 methods x 5 seeds)
%%time
all_results = {}  # {method: {seed: {callback, model, train_time}}}


def make_model(method, env, seed):
    """Create PPO model for the given method."""
    if method == 'Pure_PPO':
        policy_kwargs = dict(net_arch=dict(pi=[64, 64], vf=[64, 64]))
    elif method == 'GNS':
        if HAS_PYG:
            policy_kwargs = dict(
                features_extractor_class=GNSFeaturesExtractorV2,
                features_extractor_kwargs=dict(features_dim=64),
                net_arch=dict(pi=[64, 64], vf=[64, 64]),
            )
        else:
            policy_kwargs = dict(net_arch=dict(pi=[64, 64], vf=[64, 64]))
    elif method == 'PhysRobot_SV':
        policy_kwargs = dict(
            features_extractor_class=PhysRobotSVExtractor,
            features_extractor_kwargs=dict(features_dim=64),
            net_arch=dict(pi=[64, 64], vf=[64, 64]),
        )
    else:
        raise ValueError(f'Unknown method: {method}')

    model = PPO(
        'MlpPolicy', env, seed=seed,
        policy_kwargs=policy_kwargs,
        **PPO_KWARGS,
    )
    return model


for method in METHODS:
    all_results[method] = {}
    for seed in SEEDS:
        tag = f'{method}/seed_{seed}'
        print(f'\n{"="*60}')
        print(f'Training {tag} ({TOTAL_TIMESTEPS/1000:.0f}K steps)')
        print(f'{"="*60}')

        # Build envs
        env = DummyVecEnv([make_push_box_env(BOX_MASS_TRAIN) for _ in range(N_ENVS)])

        model = make_model(method, env, seed)
        cb = LoggingCallback()

        t0 = time.time()
        try:
            model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=cb, progress_bar=True)
            train_time = time.time() - t0
            # Save model
            model_path = f'{SAVE_DIR}/models/{method}_seed{seed}'
            model.save(model_path)

            all_results[method][seed] = {
                'ep_rewards': cb.ep_rewards,
                'ep_successes': cb.ep_successes,
                'ep_timesteps': cb.ep_timesteps,
                'train_time': train_time,
                'model_path': model_path,
            }
            n_succ = sum(cb.ep_successes)
            n_ep = len(cb.ep_successes)
            sr = n_succ / max(n_ep, 1)
            print(f'  -> Done in {train_time/60:.1f}m  |  {n_ep} episodes  |  SR={sr:.1%}')

        except Exception as e:
            train_time = time.time() - t0
            print(f'  -> FAILED after {train_time/60:.1f}m: {e}')
            import traceback; traceback.print_exc()
            all_results[method][seed] = {'error': str(e)}
        finally:
            env.close()

# Save raw data
raw_path = f'{SAVE_DIR}/results/all_training_raw.json'
_serializable = {}
for m in all_results:
    _serializable[m] = {}
    for s, d in all_results[m].items():
        if 'error' in d:
            _serializable[m][str(s)] = d
        else:
            _serializable[m][str(s)] = {
                'ep_rewards': [float(x) for x in d['ep_rewards']],
                'ep_successes': d['ep_successes'],
                'ep_timesteps': d['ep_timesteps'],
                'train_time': d['train_time'],
            }
with open(raw_path, 'w') as f:
    json.dump(_serializable, f)
print(f'\nRaw results saved -> {raw_path}')


In [ ]:
# Cell 7: Evaluation + OOD mass sweep
import numpy as np
import json

eval_results = {}  # {method: {seed: {mass: {sr, mean_rew, std_rew}}}}

for method in METHODS:
    eval_results[method] = {}
    for seed in SEEDS:
        data = all_results[method].get(seed, {})
        if 'error' in data or 'model_path' not in data:
            print(f'Skipping {method}/seed_{seed} (no model)')
            continue

        model_path = data['model_path']
        model = PPO.load(model_path)

        eval_results[method][seed] = {}
        for mass in OOD_MASSES:
            test_env = PushBoxEnv(box_mass=mass)
            rewards_list, successes_list = [], []
            for ep_seed in range(10000, 10000 + EVAL_EPISODES):
                obs, info = test_env.reset(seed=ep_seed)
                done, ep_rew = False, 0.0
                while not done:
                    action, _ = model.predict(obs, deterministic=True)
                    obs, reward, terminated, truncated, info = test_env.step(action)
                    ep_rew += reward
                    done = terminated or truncated
                rewards_list.append(ep_rew)
                successes_list.append(1 if info.get('success', False) else 0)
            test_env.close()

            sr = np.mean(successes_list)
            eval_results[method][seed][mass] = {
                'success_rate': float(sr),
                'mean_reward': float(np.mean(rewards_list)),
                'std_reward': float(np.std(rewards_list)),
            }
            tag = '[train]' if mass == BOX_MASS_TRAIN else '[OOD]'
            print(f'  {method} seed={seed} mass={mass} {tag}: SR={sr:.1%}')

# Save eval results
eval_path = f'{SAVE_DIR}/results/eval_ood_results.json'
_eval_ser = {}
for m in eval_results:
    _eval_ser[m] = {}
    for s, masses in eval_results[m].items():
        _eval_ser[m][str(s)] = {str(k): v for k, v in masses.items()}
with open(eval_path, 'w') as f:
    json.dump(_eval_ser, f, indent=2)
print(f'\nEval results saved -> {eval_path}')


In [ ]:
# Cell 8: Figures -- learning curves + OOD bar charts
import matplotlib
matplotlib.rcParams['font.size'] = 12
import matplotlib.pyplot as plt

COLORS = {'Pure_PPO': '#4CAF50', 'GNS': '#2196F3', 'PhysRobot_SV': '#FF9800'}
LABELS = {'Pure_PPO': 'Pure PPO', 'GNS': 'GNS', 'PhysRobot_SV': 'PhysRobot-SV'}

# ------- 1. Learning curves (rolling SR) -------
WINDOW = 50  # episodes

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (a) Rolling success rate vs timesteps
ax = axes[0]
for method in METHODS:
    all_curves = []
    for seed in SEEDS:
        d = all_results[method].get(seed, {})
        if 'error' in d or 'ep_successes' not in d:
            continue
        succ = np.array(d['ep_successes'], dtype=float)
        ts = np.array(d['ep_timesteps'], dtype=float)
        if len(succ) < WINDOW:
            continue
        # Rolling average
        roll = np.convolve(succ, np.ones(WINDOW)/WINDOW, mode='valid')
        roll_ts = ts[WINDOW-1:]
        all_curves.append((roll_ts, roll))

    if not all_curves:
        continue

    # Interpolate to common x-grid for mean/std
    x_grid = np.linspace(0, TOTAL_TIMESTEPS, 200)
    interp_curves = []
    for ts_c, roll_c in all_curves:
        interp_y = np.interp(x_grid, ts_c, roll_c, left=roll_c[0], right=roll_c[-1])
        interp_curves.append(interp_y)
    interp_curves = np.array(interp_curves)
    mean_curve = interp_curves.mean(axis=0)
    std_curve = interp_curves.std(axis=0)

    color = COLORS[method]
    ax.plot(x_grid / 1000, mean_curve * 100, label=LABELS[method], color=color, linewidth=2)
    ax.fill_between(
        x_grid / 1000,
        (mean_curve - std_curve) * 100,
        (mean_curve + std_curve) * 100,
        alpha=0.2, color=color,
    )

ax.set_xlabel('Timesteps (K)')
ax.set_ylabel('Success Rate (%)')
ax.set_title(f'Learning Curves (rolling {WINDOW}-ep SR, mean +/- std over {len(SEEDS)} seeds)')
ax.legend()
ax.set_ylim(-5, 105)
ax.grid(True, alpha=0.3)

# (b) Rolling episode reward
ax = axes[1]
for method in METHODS:
    all_curves = []
    for seed in SEEDS:
        d = all_results[method].get(seed, {})
        if 'error' in d or 'ep_rewards' not in d:
            continue
        rews = np.array(d['ep_rewards'], dtype=float)
        ts = np.array(d['ep_timesteps'], dtype=float)
        if len(rews) < WINDOW:
            continue
        roll = np.convolve(rews, np.ones(WINDOW)/WINDOW, mode='valid')
        roll_ts = ts[WINDOW-1:]
        all_curves.append((roll_ts, roll))

    if not all_curves:
        continue

    x_grid = np.linspace(0, TOTAL_TIMESTEPS, 200)
    interp_curves = []
    for ts_c, roll_c in all_curves:
        interp_y = np.interp(x_grid, ts_c, roll_c, left=roll_c[0], right=roll_c[-1])
        interp_curves.append(interp_y)
    interp_curves = np.array(interp_curves)
    mean_curve = interp_curves.mean(axis=0)
    std_curve = interp_curves.std(axis=0)

    color = COLORS[method]
    ax.plot(x_grid / 1000, mean_curve, label=LABELS[method], color=color, linewidth=2)
    ax.fill_between(
        x_grid / 1000,
        mean_curve - std_curve,
        mean_curve + std_curve,
        alpha=0.2, color=color,
    )

ax.set_xlabel('Timesteps (K)')
ax.set_ylabel('Episode Reward')
ax.set_title(f'Learning Curves (rolling {WINDOW}-ep reward, mean +/- std)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/learning_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: learning_curves.png')


# ------- 2. OOD Bar Charts -------
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (a) In-distribution SR comparison (bar chart)
ax = axes[0]
bar_data = []
for method in METHODS:
    srs = []
    for seed in SEEDS:
        if seed in eval_results.get(method, {}):
            sr = eval_results[method][seed].get(BOX_MASS_TRAIN, {}).get('success_rate', 0)
            srs.append(sr)
    if srs:
        bar_data.append((method, np.mean(srs), np.std(srs)))
    else:
        bar_data.append((method, 0, 0))

x_pos = np.arange(len(bar_data))
means = [d[1] * 100 for d in bar_data]
stds = [d[2] * 100 for d in bar_data]
colors = [COLORS[d[0]] for d in bar_data]
bars = ax.bar(x_pos, means, yerr=stds, capsize=5, color=colors, alpha=0.85)
ax.set_xticks(x_pos)
ax.set_xticklabels([LABELS[d[0]] for d in bar_data])
ax.set_ylabel('Success Rate (%)')
ax.set_title(f'In-Distribution SR (mass={BOX_MASS_TRAIN}, {len(SEEDS)} seeds)')
ax.set_ylim(0, 105)
for bar, m in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width() / 2., bar.get_height() + 2,
            f'{m:.1f}%', ha='center', va='bottom', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# (b) OOD SR vs mass (line plot)
ax = axes[1]
for method in METHODS:
    mass_means = []
    mass_stds = []
    for mass in OOD_MASSES:
        srs = []
        for seed in SEEDS:
            if seed in eval_results.get(method, {}):
                sr = eval_results[method][seed].get(mass, {}).get('success_rate', 0)
                srs.append(sr)
        if srs:
            mass_means.append(np.mean(srs))
            mass_stds.append(np.std(srs))
        else:
            mass_means.append(0)
            mass_stds.append(0)

    mass_means = np.array(mass_means) * 100
    mass_stds = np.array(mass_stds) * 100
    color = COLORS[method]
    ax.errorbar(
        OOD_MASSES, mass_means, yerr=mass_stds,
        fmt='o-', label=LABELS[method], color=color,
        linewidth=2, markersize=8, capsize=4,
    )

ax.axvline(x=BOX_MASS_TRAIN, color='gray', linestyle='--', alpha=0.5, label='Training mass')
ax.set_xlabel('Box Mass (kg)')
ax.set_ylabel('Success Rate (%)')
ax.set_title(f'OOD Generalization (mass sweep, {len(SEEDS)} seeds)')
ax.legend()
ax.set_ylim(-5, 105)
ax.grid(True, alpha=0.3)

plt.tight_layout()
fig.savefig(f'{SAVE_DIR}/figures/ood_bar_charts.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: ood_bar_charts.png')


# ------- 3. Summary table -------
print('\n' + '=' * 70)
print('SUMMARY TABLE')
print('=' * 70)
header = f'{"Method":<16} {"ID SR (%)":<12} {"OOD-1.0 (%)":<12} {"OOD-2.0 (%)":<12} {"Time (min)":<12}'
print(header)
print('-' * 70)
for method in METHODS:
    id_srs, ood1_srs, ood2_srs, times = [], [], [], []
    for seed in SEEDS:
        d = all_results[method].get(seed, {})
        if 'train_time' in d:
            times.append(d['train_time'])
        if seed in eval_results.get(method, {}):
            id_srs.append(eval_results[method][seed].get(BOX_MASS_TRAIN, {}).get('success_rate', 0))
            ood1_srs.append(eval_results[method][seed].get(1.0, {}).get('success_rate', 0))
            ood2_srs.append(eval_results[method][seed].get(2.0, {}).get('success_rate', 0))

    def fmt(arr):
        if not arr:
            return 'N/A'
        return f'{np.mean(arr)*100:.1f} +/- {np.std(arr)*100:.1f}'

    t_str = f'{np.mean(times)/60:.1f}' if times else 'N/A'
    print(f'{LABELS[method]:<16} {fmt(id_srs):<12} {fmt(ood1_srs):<12} {fmt(ood2_srs):<12} {t_str:<12}')


In [ ]:
# Cell 9: Save everything to Drive
import shutil

# Final summary JSON
summary = {
    'config': {
        'methods': METHODS,
        'seeds': SEEDS,
        'total_timesteps': TOTAL_TIMESTEPS,
        'n_envs': N_ENVS,
        'box_mass_train': BOX_MASS_TRAIN,
        'eval_episodes': EVAL_EPISODES,
        'ood_masses': OOD_MASSES,
        'ppo_kwargs': {k: v for k, v in PPO_KWARGS.items() if k != 'verbose'},
    },
    'eval_results': _eval_ser,
}
summary_path = f'{SAVE_DIR}/results/phase1_ablation_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print('=== Files saved to Google Drive ===')
for root, dirs, files in os.walk(SAVE_DIR):
    level = root.replace(SAVE_DIR, '').count(os.sep)
    indent = '  ' * level
    print(f'{indent}{os.path.basename(root)}/')
    sub_indent = '  ' * (level + 1)
    for f in sorted(files):
        fpath = os.path.join(root, f)
        size_kb = os.path.getsize(fpath) / 1024
        print(f'{sub_indent}{f}  ({size_kb:.1f} KB)')

print(f'\nDone. All results at: {SAVE_DIR}')
